In [80]:
!pip install open_clip_torch

import os
import json
import random
import torch
import open_clip
from PIL import Image
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity

DEPRECATION: Loading egg at /shared/home/kyscap251/miniconda3/lib/python3.12/site-packages/pycocoevalcap-1.2-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /shared/home/kyscap251/miniconda3/lib/python3.12/site-packages/six-1.17.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /shared/home/kyscap251/miniconda3/lib/python3.12/site-packages/contourpy-1.3.2-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /shared/home/kyscap251/miniconda3/l

In [81]:
# 기본 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

# OpenCLIP 모델 및 전처리 로딩 (사전학습된 모델 사용)
model, _, preprocess = open_clip.create_model_and_transforms(
    'ViT-B-32',
    pretrained='laion2b_s34b_b79k'  # 또는 'openai' 등
)
model.to(device)
model.float()


Device: cuda


CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-11): 12 x ResidualAttentionBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((768,), eps=1e-05, elementwise_affine

In [82]:
# COCO 이미지 및 캡션 경로 설정
image_dir = "train2017"
caption_json_path = "annotations/captions_train2017.json"

# 캡션 로드 및 매핑
with open(caption_json_path, 'r') as f:
    coco_data = json.load(f)

imgid_to_captions = defaultdict(list)
for ann in coco_data['annotations']:
    imgid_to_captions[ann['image_id']].append(ann['caption'])

In [83]:
# 이미지 3장 샘플링
image_list = sorted([f for f in os.listdir(image_dir) if f.endswith(".jpg")])
sample_images = random.sample(image_list, 3)

# 샘플 이미지 3장, 각 이미지에 대해 caption 5개 추출
image_paths = []
captions_list = []

for fname in sample_images:
    image_id = int(os.path.splitext(fname)[0])
    cap_list = imgid_to_captions.get(image_id, [])
    if cap_list:
        image_paths.append(os.path.join(image_dir, fname))
        # 최대 5개 캡션 사용
        captions_list.append(cap_list[:5])


In [84]:
print("샘플 이미지:")
# 각 이미지당 대표 캡션 1개만 선택
captions = [caps[0] for caps in captions_list]
for path, cap in zip(image_paths, captions):
    print(f"- {os.path.basename(path)} → \"{cap}\"")

샘플 이미지:
- 000000376131.jpg → "A computer is used as a tool by the medical professional."
- 000000144929.jpg → "A young girl jumping into the air while holding an umbrella."
- 000000088272.jpg → "A plate containing a piece of pizza, sauce, knife and fork."


In [85]:
# 이미지 전처리 및 임베딩
images = torch.stack([preprocess(Image.open(p).convert("RGB")) for p in image_paths]).to(device)
with torch.no_grad():
    image_embeds = model.encode_image(images)
    image_embeds = image_embeds / image_embeds.norm(dim=1, keepdim=True)

# 캡션 임베딩
tokenized = tokenizer(captions).to(device)
with torch.no_grad():
    text_embeds = model.encode_text(tokenized)
    text_embeds = text_embeds / text_embeds.norm(dim=1, keepdim=True)


In [86]:
# Cosine Similarity 계산
image_embeds_np = image_embeds.cpu().numpy()
text_embeds_np = text_embeds.cpu().numpy()
sims = cosine_similarity(image_embeds_np, text_embeds_np)

# 결과 출력
print("\n OpenCLIP Cosine Similarity Matrix:")
for i, img_path in enumerate(image_paths):
    print(f"\n {os.path.basename(img_path)}")
    for j, cap in enumerate(captions):
        print(f"  \"{cap}\" → similarity: {sims[i, j]:.4f}")


 OpenCLIP Cosine Similarity Matrix:

 000000376131.jpg
  "A computer is used as a tool by the medical professional." → similarity: 0.2595
  "A young girl jumping into the air while holding an umbrella." → similarity: -0.0219
  "A plate containing a piece of pizza, sauce, knife and fork." → similarity: 0.0491

 000000144929.jpg
  "A computer is used as a tool by the medical professional." → similarity: 0.0451
  "A young girl jumping into the air while holding an umbrella." → similarity: 0.2786
  "A plate containing a piece of pizza, sauce, knife and fork." → similarity: 0.0012

 000000088272.jpg
  "A computer is used as a tool by the medical professional." → similarity: 0.0679
  "A young girl jumping into the air while holding an umbrella." → similarity: 0.0563
  "A plate containing a piece of pizza, sauce, knife and fork." → similarity: 0.3487


In [87]:
# 이미지-텍스트 쌍 (정확히 매칭된 1:1 쌍만)
captions = [caps[0] for caps in captions_list]  # 각 이미지의 첫 번째 캡션을 정답으로 사용

# 이미지 임베딩
images = torch.stack([preprocess(Image.open(p).convert("RGB")) for p in image_paths]).to(device).float()
with torch.no_grad():
    image_embeds = model.encode_image(images)
    image_embeds = image_embeds / image_embeds.norm(dim=1, keepdim=True)

# 텍스트 임베딩
tokenized = tokenizer(captions).to(device)
with torch.no_grad():
    text_embeds = model.encode_text(tokenized)
    text_embeds = text_embeds / text_embeds.norm(dim=1, keepdim=True)

# Cosine similarity 계산
sims = cosine_similarity(image_embeds.cpu().numpy(), text_embeds.cpu().numpy())

# 정답 쌍 유사도만 출력
print("\n정확히 매칭된 쌍 유사도:")
for i in range(len(image_paths)):
    print(f"{os.path.basename(image_paths[i])} ⟶ \"{captions[i]}\" → similarity: {sims[i, i]:.4f}")



정확히 매칭된 쌍 유사도:
000000376131.jpg ⟶ "A computer is used as a tool by the medical professional." → similarity: 0.2595
000000144929.jpg ⟶ "A young girl jumping into the air while holding an umbrella." → similarity: 0.2786
000000088272.jpg ⟶ "A plate containing a piece of pizza, sauce, knife and fork." → similarity: 0.3487


In [88]:
import torch
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
import os

# 1. 이미지 임베딩
images = torch.stack([
    preprocess(Image.open(p).convert("RGB")) for p in image_paths
]).to(device).float()

with torch.no_grad():
    image_embeds = model.encode_image(images)
    image_embeds = image_embeds / image_embeds.norm(dim=1, keepdim=True)

# 2. 각 이미지에 대해 5개 캡션 중 best one 선택
best_text_feats = []
best_captions = []

for i, caps in enumerate(captions_list):  # captions_list[i] : i번째 이미지의 5개 캡션
    tokenized = tokenizer(caps).to(device)
    with torch.no_grad():
        text_embeds_5 = model.encode_text(tokenized)
        text_embeds_5 = text_embeds_5 / text_embeds_5.norm(dim=1, keepdim=True)

        # 이미지 i번째와 5개 텍스트 임베딩 간 cosine 유사도 계산
        sims = torch.matmul(image_embeds[i].unsqueeze(0), text_embeds_5.T).squeeze(0)
        best_idx = sims.argmax().item()
        best_text_feats.append(text_embeds_5[best_idx].unsqueeze(0))
        best_captions.append(caps[best_idx])  # 해당 캡션 저장

text_embeds = torch.cat(best_text_feats, dim=0)

# 3. Cosine similarity (diagonal만)
sims = cosine_similarity(image_embeds.cpu().numpy(), text_embeds.cpu().numpy())

# 4. 출력
print("\nOpenCLIP Cosine Similarity (Best-of-5 Captions):")
for i in range(len(image_paths)):
    print(f"\n {os.path.basename(image_paths[i])}")
    print(f" Best caption: \"{best_captions[i]}\"")
    print(f" similarity: {sims[i, i]:.4f}")



OpenCLIP Cosine Similarity (Best-of-5 Captions):

 000000376131.jpg
 Best caption: "A doctor's office counter cover in papers and a laptop"
 similarity: 0.3775

 000000144929.jpg
 Best caption: "A young girl wearing shorts leaps into the air while holding an umbrella."
 similarity: 0.2960

 000000088272.jpg
 Best caption: "A piece of pizza and mustard are on a plate."
 similarity: 0.3487
